<a href="https://colab.research.google.com/github/Aerospace87/ML-projects/blob/main/tensorflow/Text_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
from google.colab import drive

# Mount your Google Drive to the local /content/drive directory
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
from os import listdir, path

local_folder = "/content/drive/My Drive/"

with open(path.join(local_folder ,"dataset/shakespeare.txt")) as f:
  shakespeare_dataset = f.read()

## Cleaning the dataset and spliting into training examples and lables

In [84]:
shakespeare_dataset = shakespeare_dataset.split('\n\n')
shakespeare_dataset = shakespeare_dataset[2:-1]

In [85]:
sos_token = "<SOS>"
eos_token = "<EOS>"
space = " "

dataset = [[sos_token + space + document ] for document in shakespeare_dataset if document != ""]
labels = [[document + space + eos_token] for document in shakespeare_dataset if document != ""]

number_documents = len(dataset)
print(number_documents)

141


In [86]:
max_doc_length = 0

for doc in dataset:
  if len(doc[0].split()) > max_doc_length:

    max_doc_length = len(doc[0].split())

print(max_doc_length)

238


In [87]:
from tensorflow.keras import layers
text_vectorization = layers.TextVectorization(
    standardize='lower',
    split='whitespace',
    output_mode='int'
)


## Text Vectorization

In [88]:
text_vectorization.adapt(dataset)

In [89]:
dataset = text_vectorization(dataset)
vocabulary_size = text_vectorization.vocabulary_size()
vocabulary = text_vectorization.get_vocabulary()

In [90]:
# Dictionay to map back from integer index to word
from_int_to_word ={}
from_word_to_int = {}

for idx,word in enumerate(vocabulary):
    from_int_to_word[idx] = word
    from_word_to_int[word] = idx

In [91]:
labels = text_vectorization(labels)

## Creation of a tensorflow dataset using training examples and labels

In [92]:
from tensorflow.data import Dataset
# Creation of the dataset to feed to the model
dataset = Dataset.from_tensor_slices(
    (dataset, labels)
)

# shuffle and split into batches of batch size BATCH_SIZE
BATCH_SIZE = 64

# Buffer size of the buffer to shuffle the dataset
# Otherfwise TF will have the full dataset in memory
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Model Creation

In [93]:
word_embedding_vector_dimension = 16

word_embedding = layers.Embedding(
    input_dim = vocabulary_size,
    output_dim=word_embedding_vector_dimension
)

In [94]:
lstm_layer_1 = layers.LSTM(128, input_shape = (1 ,number_documents, word_embedding_vector_dimension), return_sequences = True)
lstm_layer_2 = layers.LSTM(128, return_sequences = True)

In [95]:
from tensorflow.keras.activations import softmax
softmax_layer = layers.Dense(units=vocabulary_size, activation=softmax)

In [96]:
from tensorflow.keras import Sequential

steps = [word_embedding, lstm_layer_1, lstm_layer_2, softmax_layer]
model = Sequential(steps)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 16)          68720     
                                                                 
 lstm_8 (LSTM)               (None, None, 128)         74240     
                                                                 
 lstm_9 (LSTM)               (None, None, 128)         131584    
                                                                 
 dense_4 (Dense)             (None, None, 4295)        554055    
                                                                 
Total params: 828599 (3.16 MB)
Trainable params: 828599 (3.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
loss = SparseCategoricalCrossentropy(from_logits=True)

In [98]:
from tensorflow.keras.optimizers import Adam
adam_optimizer = Adam(learning_rate=1e-3)

In [99]:
model.compile(optimizer = adam_optimizer, loss = loss)

## Testing the model works

In [100]:
first_batch = dataset.take(1)

In [101]:
for input_data, labels in first_batch:
  predictions = model(input_data)
  print(predictions.shape)

(64, 238, 4295)


## Configuring checkpoints

In [102]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [103]:
checkpoint_folder = path.join(local_folder,
                              "tf_checkpoints/Text Generation using RNN"
)

checkpoint_files_path = path.join(checkpoint_folder, "ckpt_{epoch}")

EPOCHS = 10

checkpoint_callback = ModelCheckpoint(
    filepath = checkpoint_files_path,
    save_weights_only=True
)

## Taining

In [108]:
restart = True
restart_epoch = 10

if restart:
  checkpoint_files_path = path.join(checkpoint_folder, f"ckpt_{restart_epoch}")
  model.load_weights(checkpoint_files_path)

model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
3/3 [==============================] - 7s 2s/step - loss: 4.2474
Epoch 2/10
3/3 [==============================] - 8s 2s/step - loss: 4.2472
Epoch 3/10
3/3 [==============================] - 7s 2s/step - loss: 4.2448
Epoch 4/10
3/3 [==============================] - 8s 2s/step - loss: 4.2412
Epoch 5/10
3/3 [==============================] - 6s 2s/step - loss: 4.2384
Epoch 6/10
3/3 [==============================] - 8s 2s/step - loss: 4.2375
Epoch 7/10
3/3 [==============================] - 6s 2s/step - loss: 4.2376
Epoch 8/10
3/3 [==============================] - 8s 2s/step - loss: 4.2368
Epoch 9/10
3/3 [==============================] - 8s 3s/step - loss: 4.2361
Epoch 10/10
3/3 [==============================] - 7s 2s/step - loss: 4.2351


In [105]:
model